In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import xml.etree.ElementTree as ET

In [2]:
DATA_PATH = Path('data')
MDB_XML_PATH = DATA_PATH / 'MDB_STAMMDATEN.XML'
DF_MDB_PATH = DATA_PATH / 'df_mdb.csv'

# parse xml data into dataFrame / .csv fomat

In [3]:
def get_mdb_data(mdb_xml_path = MDB_XML_PATH):
    list_of_mdb = []
    MDB_STAMMDATEN = ET.parse(mdb_xml_path)
    root = MDB_STAMMDATEN.getroot()
    
    for mdb in root.findall('MDB'):
        mdb_dict = {}
        ID, NAMEN, BIOGRAFISCHE_ANGABEN, WAHLPERIODEN = mdb[0], mdb[1], mdb[2], mdb[3]

        mdb_dict['ID'] = ID.text
        mdb_dict['NACHNAME'] = NAMEN.find('NAME').find('NACHNAME').text  
        mdb_dict['VORNAME'] = NAMEN.find('NAME').find('VORNAME').text  

        #print(mdb.tag, mdb.text, mdb.findall('NACHNAME'))
        for element in BIOGRAFISCHE_ANGABEN:
            mdb_dict[element.tag] = element.text

        mdb_dict['ANZ_WAHLPERIODEN'] = len(list(WAHLPERIODEN))

        for wahlperiode in WAHLPERIODEN:
            mdb_dict[int(wahlperiode.find('WP').text)] = 1

        list_of_mdb.append(mdb_dict)
    return pd.DataFrame(list_of_mdb)

In [4]:
df_mdb = get_mdb_data(MDB_XML_PATH)
print(df_mdb.PARTEI_KURZ)
print(df_mdb.shape)
df_mdb.head()

0                         CDU
1                         FDP
2                         CDU
3                         CDU
4                         CDU
                ...          
4084                      FDP
4085                      SPD
4086    BÜNDNIS 90/DIE GRÜNEN
4087                      CDU
4088    BÜNDNIS 90/DIE GRÜNEN
Name: PARTEI_KURZ, Length: 4089, dtype: object
(4089, 34)


,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11000002,Achenbach,Ernst,09.04.1909,Siegen,None,02.12.1991,männlich,"verheiratet, 3 Kinder",evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11000004,Ackermann,Else,06.11.1933,Berlin,None,14.09.2019,weiblich,ledig,evangelisch,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000005,Adam,Ulrich,09.06.1950,"Teterow, Kr. Teterow, Bezirk Neubrandenburg",None,None,männlich,"verheiratet, 2 Kinder",evangelisch,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


# clean data

In [5]:
# ok, quite expected. some cleaning necessarry: 'BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE'
for grünen_alias in ['BÜNDNIS 90/DIE GRÜNEN', 'DIE GRÜNEN/BÜNDNIS 90', 'GRÜNE']:
    df_mdb.replace(grünen_alias, 'DIE GRÜNEN', inplace=True)

# save as csv

In [6]:
df_mdb.to_csv(DF_MDB_PATH, index=None)

# read in data. exploratory data analysis

In [7]:
df_mdb = pd.read_csv(DF_MDB_PATH, index_col=None)

In [8]:
df_mdb.columns

Index(['ID', 'NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'GEBURTSORT',
       'GEBURTSLAND', 'STERBEDATUM', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION',
       'BERUF', 'PARTEI_KURZ', 'VITA_KURZ', 'VEROEFFENTLICHUNGSPFLICHTIGES',
       'ANZ_WAHLPERIODEN', '5', '6', '7', '8', '9', '10', '11', '3', '4', '2',
       '12', '13', '14', '15', '16', '1', '17', '18', '19'],
      dtype='object')

In [9]:
df_mdb.tail()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
4084,11004958,Nölke,Matthias,08.01.1980,Kassel,NaN,NaN,männlich,ledig,konfessionslos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4085,11004959,Martin,Dorothee,21.01.1978,NaN,NaN,NaN,weiblich,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4086,11004960,Wetzel,Wolfgang,11.05.1968,Schlema,NaN,NaN,männlich,"verheiratet, 2 Kinder",römisch-katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4087,11004961,Natterer,Christian,19.02.1981,Tettnang / Deutschland,NaN,NaN,männlich,"verheiratet, 1 Kind",römisch-katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4088,11004962,Dahmen,Janosch,06.09.1981,NaN,NaN,NaN,männlich,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [10]:
# deleteme
selected_df = pd.concat([df_mdb[df_mdb[str(i)] == 1] for i in range(14,16)]).drop_duplicates()
selected_df = df_mdb[(df_mdb['14'] == 1) |(df_mdb['15'] ==1) ]
selected_df.shape

(897, 34)

In [11]:
df_mdb[['ID', 'PARTEI_KURZ']].groupby('PARTEI_KURZ').count().sort_values(by='ID', ascending=False).head(10)

,ID
PARTEI_KURZ,
CDU,1347
SPD,1329
FDP,460
CSU,269
DIE GRÜNEN,259
DIE LINKE.,128
AfD,88
PDS,54
DP,27


In [12]:
# wie - nur 2 Geschlechter? for the records: heute ist der 02.05.2021
df_mdb[['ID', 'GESCHLECHT']].groupby('GESCHLECHT').count().sort_values(by='ID', ascending=False).head()

,ID
GESCHLECHT,
männlich,3231
weiblich,858


In [13]:
# auch hier: such diversity. wow
df_mdb[['ID', 'RELIGION']].groupby('RELIGION').count().sort_values(by='ID', ascending=False).head(n=16)

,ID
RELIGION,
ohne Angaben,1214
evangelisch,1154
katholisch,870
römisch-katholisch,316
evangelisch-lutherisch,138
konfessionslos,118
evangelisch-reformiert,15
religionslos,7
Atheistin,6


In [14]:
# so viele Busfahrer und Krankenschwestern!
df_mdb[['ID', 'BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head(16).head(16)

,ID
BERUF,
Rechtsanwalt,182
Geschäftsführer,80
Landwirt,79
Angestellter,63
Rechtsanwalt und Notar,47
Kaufmann,45
Journalist,43
Hausfrau,41
Dipl.-Ingenieur,37


In [15]:
# gehen sie weiter. hier gibt es kein Klischee zu erkennen!
df_mdb[['ID', 'FAMILIENSTAND']].groupby('FAMILIENSTAND').count().sort_values(by='ID', ascending=False).head(16)

,ID
FAMILIENSTAND,
"verheiratet, 2 Kinder",932
verheiratet,565
"verheiratet, 3 Kinder",550
keine Angaben,540
"verheiratet, 1 Kind",465
"verheiratet, 4 Kinder",251
ledig,202
"verheiratet, 5 Kinder",98
"verheiratet, 6 Kinder",47


In [30]:
# deleteme
start_date=2
end_date=4
wps = range(start_date, end_date)
selected_df = pd.concat([df_mdb[df_mdb[str(i)] == 1] for i in range(start_date,end_date+1)]).drop_duplicates()# deleteme
selected_df.head()

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11000009,Adenauer,Konrad,05.01.1876,Köln,NaN,19.04.1967,männlich,keine Angaben,katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
18,11000020,Albers,Johannes,08.03.1890,Mönchengladbach,NaN,08.03.1963,männlich,keine Angaben,katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
19,11000021,Albertz,Luise,22.06.1901,Duisburg,NaN,01.02.1979,weiblich,keine Angaben,ohne Angaben,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
21,11000023,Albrecht,Ernst,28.07.1914,Greifswald,NaN,01.12.1991,männlich,verheiratet,evangelisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# deleteme
x =  selected_df[['ID', 'GESCHLECHT']].groupby('GESCHLECHT').count().sort_values(by='ID', ascending=False).index
selected_parteien = ['CDU', 'SPD']
selected_df = selected_df[selected_df['PARTEI_KURZ'].isin(selected_parteien)]
selected_df

,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11000009,Adenauer,Konrad,05.01.1876,Köln,NaN,19.04.1967,männlich,keine Angaben,katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
18,11000020,Albers,Johannes,08.03.1890,Mönchengladbach,NaN,08.03.1963,männlich,keine Angaben,katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
19,11000021,Albertz,Luise,22.06.1901,Duisburg,NaN,01.02.1979,weiblich,keine Angaben,ohne Angaben,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
21,11000023,Albrecht,Ernst,28.07.1914,Greifswald,NaN,01.12.1991,männlich,verheiratet,evangelisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,11002469,Wellmann,Hans,06.09.1911,Berlin,NaN,30.05.1969,männlich,"verheiratet, 2 Kinder",ohne Angaben,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2512,11002529,Winterstein,Ladislaus,11.06.1905,Altsiwatz,Jugoslawien,02.11.1964,männlich,"verheiratet, 3 Kinder",ohne Angaben,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2537,11002554,Wolf,Willi,24.02.1924,Essen,NaN,30.09.2007,männlich,"verheiratet, 3 Kinder",ohne Angaben,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2578,11002596,Zimmermann,Else,14.08.1907,Mönchengladbach,NaN,21.06.1995,weiblich,"verheiratet, 2 Kinder",ohne Angaben,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# deleteme
page_current = 0
page_size = 4
COLUMNS_FOR_DISPLAY = ['NACHNAME', 'VORNAME', 'GEBURTSDATUM', 'PARTEI_KURZ', 'GESCHLECHT', 'FAMILIENSTAND', 'RELIGION', 'BERUF', 'VITA_KURZ']
    
selected_df.iloc[
        page_current*page_size:(page_current+ 1)*page_size
    ][COLUMNS_FOR_DISPLAY].to_dict('records')

[{'NACHNAME': 'Ackermann',
  'VORNAME': 'Annemarie',
  'GEBURTSDATUM': '26.05.1913',
  'PARTEI_KURZ': 'CDU',
  'GESCHLECHT': 'weiblich',
  'FAMILIENSTAND': 'verheiratet, 5 Kinder',
  'RELIGION': 'katholisch',
  'BERUF': 'Hilfsreferentin',
  'VITA_KURZ': nan},
 {'NACHNAME': 'Adenauer',
  'VORNAME': 'Konrad',
  'GEBURTSDATUM': '05.01.1876',
  'PARTEI_KURZ': 'CDU',
  'GESCHLECHT': 'männlich',
  'FAMILIENSTAND': 'keine Angaben',
  'RELIGION': 'katholisch',
  'BERUF': 'Bundeskanzler a. D.',
  'VITA_KURZ': nan},
 {'NACHNAME': 'Albers',
  'VORNAME': 'Johannes',
  'GEBURTSDATUM': '08.03.1890',
  'PARTEI_KURZ': 'CDU',
  'GESCHLECHT': 'männlich',
  'FAMILIENSTAND': 'keine Angaben',
  'RELIGION': 'katholisch',
  'BERUF': 'Geschäftsführer',
  'VITA_KURZ': nan},
 {'NACHNAME': 'Albertz',
  'VORNAME': 'Luise',
  'GEBURTSDATUM': '22.06.1901',
  'PARTEI_KURZ': 'SPD',
  'GESCHLECHT': 'weiblich',
  'FAMILIENSTAND': 'keine Angaben',
  'RELIGION': 'ohne Angaben',
  'BERUF': 'Buchhalterin',
  'VITA_KURZ': n